In [ ]:
# komórka na przygotowanie bibliotek, obrazków, etc.
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

# Ustawienie rozmarów wyświetlanych obrazów
plt.rcParams["figure.figsize"] = (18, 10)

image = cv.imread("images/example.jpg")
image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

# Zadanie 1

In [ ]:
kernel = [
    [-1, -1, -1],
    [-1, 8, -1],
    [-1, -1, -1],
]
kernel = np.asarray(kernel)
filtered_image = cv.filter2D(image, -1, kernel=kernel) 

plt.imshow(filtered_image)

# Zadanie 2

In [ ]:
changed_image = image.astype(np.float32)
changed_image /= 255.
matrix = np.array([0.393, 0.769, 0.189, 0.349, 0.689, 0.168, 0.272, 0.534, 0.131]).reshape(3, 3)

# https://stackoverflow.com/a/22081723, jeśli jest jakiś mądrzejszy sposób to chętnie się dowiem (wolałem unikać pętli)
reshaped_image = changed_image.reshape((changed_image.shape[0] * changed_image.shape[1], changed_image.shape[2]))
result_image = np.clip(np.dot(matrix, reshaped_image.T).T.reshape(changed_image.shape), 0, 1)

plt.imshow(result_image)

# Zadanie 3

In [ ]:
mult_matrix = np.array([0.229, 0.587, 0.114, 0.500, -0.418, -0.082, -0.168, -0.331, 0.500]).reshape(3, 3)
add_matrix = np.array([0, 128, 128]).astype(np.float32).reshape(3, 1)  # float32 żeby można było dodać

# reshape obrazka
reshaped_image = image.reshape((image.shape[0] * image.shape[1], image.shape[2]))

# mnożenie przez macierz
yCrCb_image = np.dot(mult_matrix, reshaped_image.T).T.astype(np.float32)  # explicit typ żeby można było dodać

# dodawanie macierzy
for i in range(len(yCrCb_image)):
    yCrCb_image[i] = cv.add(add_matrix, yCrCb_image[i].T).T

# reshape z powrotem
yCrCb_image = np.array(yCrCb_image).astype(np.uint8).reshape(image.shape)


fig, ax = plt.subplots(2, 3)

ax[0, 0].imshow(image)
ax[0, 0].set_xlabel('Oryginalny obraz')

ax[0, 1].imshow(yCrCb_image)
ax[0, 1].set_xlabel('Obraz po konwersji na YCrCb')

ax[0, 2].imshow(cv.cvtColor(yCrCb_image, cv.COLOR_YCrCb2RGB))
ax[0, 2].set_xlabel('Konwersja odwrotna')

ax[1, 0].imshow(yCrCb_image[:, :, 0], cmap="Greys_r")
ax[1, 0].set_xlabel('Składowa Y')

ax[1, 1].imshow(yCrCb_image[:, :, 1], cmap="Greys_r")
ax[1, 1].set_xlabel('Składowa Cr')

ax[1, 2].imshow(yCrCb_image[:, :, 2], cmap="Greys_r")
ax[1, 2].set_xlabel('Składowa Cb')

# Zadanie 4

In [ ]:
y_component, cr, cb = cv.split(yCrCb_image)

downsampled_cr = np.empty((len(cr) // 2, len(cr[0]) // 2)).astype(np.uint8)
downsampled_cb = np.empty((len(cr) // 2, len(cr[0]) // 2)).astype(np.uint8)

for x in range(len(cr) // 2):
    for y in range(len(cr[0]) // 2):
        downsampled_cr[x][y] = cr[x * 2][y * 2]
        downsampled_cb[x][y] = cb[x * 2][y * 2]

upsampled_cr = np.empty((len(cr), len(cr[0]))).astype(np.uint8)
upsampled_cb = np.empty((len(cr), len(cr[0]))).astype(np.uint8)

for x in range(len(cr) // 2):
    for y in range(len(cr[0]) // 2):
        upsampled_cr[x * 2][y * 2] = downsampled_cr[x][y]
        upsampled_cr[x * 2 + 1][y * 2] = downsampled_cr[x][y]
        upsampled_cr[x * 2][y * 2 + 1] = downsampled_cr[x][y]
        upsampled_cr[x * 2 + 1][y * 2 + 1] = downsampled_cr[x][y]

        upsampled_cb[x][y] = downsampled_cb[x][y]
        upsampled_cb[x + 1][y] = downsampled_cb[x][y]
        upsampled_cb[x][y + 1] = downsampled_cb[x][y]
        upsampled_cb[x + 1][y + 1] = downsampled_cb[x][y]

yCrCb_upsampled_image = np.empty((len(cr), len(cr[0]), 3)).astype(np.uint8)

cv.merge((y_component.astype(np.uint8), upsampled_cr.astype(np.uint8), upsampled_cb.astype(np.uint8)),
         yCrCb_upsampled_image)

rgb_upsampled_image = cv.cvtColor(yCrCb_upsampled_image, cv.COLOR_YCrCb2RGB)

fig, ax = plt.subplots(2, 3)

ax[0, 0].imshow(image)
ax[0, 0].set_xlabel('Oryginalny obraz')

ax[0, 1].imshow(yCrCb_image)
ax[0, 1].set_xlabel('Obraz po konwersji na YCrCb, downsamplingu i upsamplingu')

ax[0, 2].imshow(cv.cvtColor(yCrCb_image, cv.COLOR_YCrCb2RGB))
ax[0, 2].set_xlabel('Konwersja odwrotna po down i upsamplingu')

ax[1, 0].imshow(yCrCb_image[:, :, 0], cmap="Greys_r")
ax[1, 0].set_xlabel('Składowa Y')

ax[1, 1].imshow(yCrCb_image[:, :, 1], cmap="Greys_r")
ax[1, 1].set_xlabel('Nowa składowa Cr')

ax[1, 2].imshow(yCrCb_image[:, :, 2], cmap="Greys_r")
ax[1, 2].set_xlabel('Nowa składowa Cb')

# Zadanie 5

In [ ]:
mse = 0
for x in range(len(cr)):
    for y in range(len(cr[0])):
        for z in range(3):
            base_image_value = int(image[x][y][z])
            upsampling_image_value = int(rgb_upsampled_image[x][y][z])
            mse += (base_image_value - upsampling_image_value) ** 2
            # linijka wyżej - warningi w PyCharmie - overflow(???)
            # nie rozumiem czemu, chyba działa dobrze
mse /= 3 * len(cr) * len(cr[0])
print(mse)